In [1]:
import duckdb
import os
import glob
from datetime import datetime, timezone

In [2]:

# ---- CONFIG ----
## vigyazz a pathra
DB_PATH = "../databases/currency_rates.duckdb"
JSON_FOLDER = "../jsons/fx_jsons"
SCHEMA_NAME = "raw"
TABLE_NAME = "raw_ingested_data"

# 1) DB csatlakozás -- ha nincs meg db file akkor letrehoz egyet
con = duckdb.connect(DB_PATH)

In [3]:
# 7) Teszt
rowcount = con.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}").fetchone()
print("Number of rows:", rowcount[0])

Number of rows: 27


In [5]:
# 8) Teszt folyt
df = con.execute("""
    SELECT *
    FROM raw.raw_ingested_data
""").df()

df.tail()

,success,terms,privacy,timeframe,start_date,end_date,source,quotes,filename,loaded_at_utc,file_name
22,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2021-01-01,2021-12-31,USD,"{'2021-01-01': '{""USDAED"":3.673204,""USDAFN"":77...",/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...
23,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2022-01-01,2022-12-31,USD,"{'2022-01-01': '{""USDAED"":3.673042,""USDAFN"":10...",/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...
24,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2023-01-01,2023-12-31,USD,"{'2023-01-01': '{""USDAED"":3.672635,""USDAFN"":87...",/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...
25,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2024-01-01,2024-12-31,USD,"{'2024-01-01': '{""USDAED"":3.672498,""USDAFN"":70...",/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...
26,True,https://currencylayer.com/terms,https://currencylayer.com/privacy,True,2025-01-01,2025-12-01,USD,"{'2025-01-01': '{""USDAED"":3.672991,""USDAFN"":70...",/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...


In [6]:
# 9) Teszt folyt 2
df = con.execute("""
    SELECT
        filename,
        loaded_at_utc,
        q.key   AS quote_date,
        q.value AS quote_value
    FROM raw.raw_ingested_data,
        json_each(quotes) AS q
""").df()

df

,filename,loaded_at_utc,quote_date,quote_value
0,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,1999-12-31,"{""USDANG"":1.79,""USDAUD"":1.525918,""USDAWG"":1.79..."
1,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2000-12-31,"{""USDAED"":3.67246,""USDALL"":145.46158,""USDANG"":..."
2,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2001-12-31,"{""USDAED"":3.67246,""USDALL"":137.694315,""USDAMD""..."
3,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2002-12-31,"{""USDAED"":3.67246,""USDALL"":131.093735,""USDAMD""..."
4,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2003-12-31,"{""USDAED"":3.67246,""USDALL"":113.838932,""USDAMD""..."
...,...,...,...,...
9827,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2008-01-01,"{""USDAED"":3.6726,""USDALL"":82.692923,""USDAMD"":3..."
9828,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2012-01-01,"{""USDAED"":3.6728,""USDAFN"":45.736288,""USDALL"":1..."
9829,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2016-01-01,"{""USDAED"":3.672754,""USDAFN"":68.45,""USDALL"":125..."
9830,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-02T19:56:48.167235+00:00,2020-01-01,"{""USDAED"":3.673202,""USDAFN"":78.40293,""USDALL"":..."


In [7]:
# futasd a checkpointot, h a currency_rates.duckdb.wal file (Write ahead log) ami a valtoztatasokat tartalömazza eltunjön
# vegul zard be a connectiont!
con.execute("CHECKPOINT;")
con.close()